<a href="https://colab.research.google.com/github/klauss1977/ML/blob/main/Moore_Penrose_Pseudoinverse.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Pseudoinversa Moore-Penrose

$MPP$ a unei matrici $A$, notata cu $A^+$, se calculeaza cu formula:             
 **$A^+$=$V$$D^+$$U^T$** in care:                                                
  $U$, $D$ si $V$ sunt $SVD$ ale matricii $A$, cu $D^+$ obtinut prin inversarea valorilor nenule ale lui $D$

$MPP$ se foloseste pentru calcularea pseudoinversei a matricilor care nu indeplinesc conditiile pentru a putea fi inversate.

Aceste conditii sunt:

-matricea trebuie sa fie patrata (numar linii = numar coloane)

-matricea este nesingulara (nu are linii sau coloane dependente una de cealalta)